# How to Scikit-Learn

# Unsupervised Learning

## [Dimension Reduction](https://scikit-learn.org/stable/modules/decomposition.html)

### Principal Component Analysis ( [PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) )

The point is to find the successive orthogonal components that explain most of the variance of the centered data set.
Here is a very simple video on the Topic https://www.youtube.com/watch?v=FgakZw6K1QQ

you can specify in n_components
* number of features to keep
* 'mle' to let Minka's MLE algorithm fit it for you https://vismod.media.mit.edu/tech-reports/TR-514.pdf
* a percentage between 0 and 1 that represents the amount of total variance that should be explained by your features

Useful attributes
* components_ : array, shape ($n_{components}\;, n_{features}$) -- Gives you the n_components components (rows) and the contribution of each feature (columns)
* explained_variance_ (ratio_) : array, shape ($n_{components}$,) -- Gives you the variance explained by each component

Some Methods
* fit(X) : fits the model with X
* fit_transform(X) : fits AND returns the transformed data
* transform(X) : returns the transformed data using the fitted model
* inverse_transform(X) : transform your data back to the original space
* get_covariance() : computes the covariance matrix $cov \in \mathscr{M}_{n_{features}}$  
$$cov =  components^T * S^2 * components + \boldsymbol{\sigma_2} * I_{n_{features}}$$ 
where $S^2$ contains the explained variances, and $\boldsymbol{\sigma_2}$ contains the noise variances.
* get_precision() : computes the precision (inverse of the covariance)

If you're inteerested in only a certain part of the whole dataset you can use the 
* svd_solver='randomized' : it only uses the right amount of data to predict the n_features wanted

In [ ]:
from sklearn.decomposition import PCA

## X is the dataset : lines are instances, columns are features ##

pca = PCA(n_components).fit(X)
X_pca = PCA(n_components).transform(X)

X_pca = PCA(n_components).fit_transform(X)

# This function plots an elbow curve representing the variance explained by components
def plot_elbow(X,n_components=10):
    pca = PCA(n_components).fit(X)
    plt.plot(np.cumsum(pca.explained_variance_ratio_))
    plt.xlabel('number of components')
    plt.ylabel('cumulative explained variance')
    plt.title('Ratio of variance explained by the number components')
    plt.show()
    
#A more general implementation for visualizing data is available under Kernel PCA

#### [Incremental PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.IncrementalPCA.html)

For big sized data you would want to use chunks of data.
It computes estimates of components and naoise variances from a batch and then updates them with the next batch

#### [Kernel PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.KernelPCA.html)

- You can use a special kernel to separate non linear datasets : https://scikit-learn.org/stable/modules/metrics.html

    - Linear : $$ K(x,x') = x^Tx' $$
    - poly : $$ K(x,x') = ( \color {green} \gamma x^T x' + \color {blue} c_0)^\color {red}d $$
    - sigmoid : $$ K(x,x') = tanh( \color {green} \gamma x^T x' + \color {blue} c_0 ) \;\;\; $$
    - Radial basis function (RBF) : $$ K(x,x') = exp(- \color {green} \gamma \|{x-x'}\|^2) $$
    - cosine : $$ K(x,x') = \frac {x^T x'}{\|x^T\| \|x'\|} $$

You can tune some Hyper parameter

$\color {green} \gamma $ <br>
`gamma  (default = 1/n_features) is used by poly / sigmoid / rbf`<br>
$\color {blue} {c_0} $ <br>
`coef0  (default = 1)            is used by poly / sigmoid` <br>
$\color {red} d $ <br>
`degree (default = 3)            is used by poly`<br>


More info on kernels : http://crsouza.com/2010/03/17/kernel-functions-for-machine-learning-applications/

In [4]:
from sklearn.decomposition import KernelPCA

# This function plots the projection of the data on the 1 2 or 3 main components and returns the PCA
#using whichever kernel and parameter you give it

def plot_pca (X,y,kernel='linear',n_components=2,gamma=None,coef0=None,degree=None):
    pca = KernelPCA(n_components,kernel, gamma=gamma, degree=degree, coef0=coef0)
    X_pca = pca.fit_transform(X)
    print("original shape:   ", X.shape)
    print("transformed shape:", X_pca.shape)
    if n_components==1:
        plt.scatter(X_pca[:,0],np.zeros(len(X_pca),),alpha=0.2,c=y.values,vmin=-3,vmax=3,)
        plt.xlabel('Component 1')
        plt.title("data projected on the main component \n using " + kernel + " kernel")
    elif n_components==2:
        plt.scatter(X_pca[:,0],X_pca[:,1],alpha=0.2,c=y.values,vmin=-3,vmax=3)
        plt.xlabel('Component 1')
        plt.ylabel('Component 2')
        plt.title("data projected on the 2 main components \n using " + kernel + " kernel")
    elif n_components==3:
        from mpl_toolkits.mplot3d import Axes3D
        fig=plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        ax.scatter(X_pca[:,0],X_pca[:,1],X_pca[:,2],alpha=0.2,c=y.values,vmin=-3,vmax=3)
        ax.set_xlabel('Component 1')
        ax.set_ylabel('Component 2')
        ax.set_zlabel('Component 3')
        plt.title("data projected on the 3 main components \n using " + kernel + " kernel")
        return pca
    else :
        print("how am I supposed to show you that with your 2-D eyes, beta !")
        return pca
    plt.colorbar()

    plt.show()
    return pca

#### [Sparse PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.SparsePCA.html#sklearn.decomposition.SparsePCA)

You can use Sparse PCA to yield sparse component, this is used via a Lasso ($l_1$) regularization




#### [Truncated SVD](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html)

If you have a large sparse dataset that you don't want to center (because of Out Of Memory Error) use this algorithm (ex : tf-idf count matrices)




### Locally Linear Embedding ( LLE )

Manifold Learning technique : Learning locally linear space for k closest neighbor $LLE : x^{(i)} \to z^{(i)}$
- Selects k closest neighbors 
$$ K_\boldsymbol{x^{(i)}} = \underset{(\boldsymbol x^{(j)})_{j\in[[1:k]]}}{\operatorname{argmin}}
(\sum\limits_{j=1}^k d(x^{(i)}-x^{(j)})) $$
- Optimizes the weights for the locally linear relations (constructing linear model for each k subset)
$$ \boldsymbol {\hat W} = \underset{\boldsymbol W \in \mathscr M_m}{\operatorname{argmin}}
\sum\limits_{i=1}^m \| \boldsymbol {x^{(i)}} - \sum\limits_{j=1}^m w_{i,j} \boldsymbol{x^{(j)}} \|^2  $$ 
$$ \text{ where } w_{i,j}=0 \text{ if } \boldsymbol{x^{(j)}} \not\in K_\boldsymbol{x^{(i)}} 
\text{ and } \sum\limits_{j=1}^m w_{i,j}=1 $$
- Minimizes the distance between the closest neighbourg (constructing low dimensional representation)
$$ \boldsymbol {\hat Z} = \underset{\boldsymbol Z \in \mathscr M_m}{\operatorname{argmin}}
\sum\limits_{i=1}^m \| \boldsymbol {z^{(i)}} - \sum\limits_{j=1}^m \hat{w}_{i,j} \boldsymbol{z^{(j)}} \|^2  $$ 


### MultiDimensional Scaling ( MDS )



### Isomap

Creates a graph and reduces dimensionality by preserving geodesic distance

### t-Distributed Stochastic Neighbor Embedding ( t-SNE )



### Linear Discriminant Analysis ( LDA )



### Independent Component Analysis ( ICA )

### Non-Negative Matrix Factorization ( NMF )

## [Clustering](https://scikit-learn.org/stable/modules/clustering.html)

### [K-means](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html)

![K-means algorithm example](https://cdn-images-1.medium.com/max/800/1*KrcZK0xYgTa4qFrVr0fO2w.gif)

K-means algorithm and [a nice visualization](https://www.naftaliharris.com/blog/visualizing-k-means-clustering/) :
   1. Arbitrarly defining K centroids points
   2. Labelling each point according to their closest centroid
   3. Putting the new centroid at the mean of all the points of each cluster (same label) points
   4. Repeating 2 and 3 until the centroids converge <br>
Complexity = $O(n)$

Useful Parameters :
- n_clusters (int) : K of the K means
- init : "[k-means++](https://en.wikipedia.org/wiki/K-means%2B%2B)" (default), "random" if you want to initialize your centroids using random points, or an array of initializing points
- n_init (int) : number of times the k means will be run with new initial centroids
- max_iter (int) : number of repetition of 2 and 3
- tol (float) : limit to declare the convergence of the centroid $(if \ inertia_{t+1}-inertia_{t}<tol)$
- n_jobs (int) : lets you define how many CPU you want to use for computation, -1 means all, None (default) is equal to 1 by default

Attributes :
- cluster_centers_ (array) : gives the position of all your clusters, shape : [$n_{clusters}$ \, $n_{features}$]
- labels_ (list) : Labels of each point given to fit()
- inertia_ (float) : Sum of squared distances of samples to their closest cluster center
- n_iter_ (int) : Number of iterations run.

Methods :
- fit(X) : Runs the algorithm to find the clusters
- transform(X) : Transforms your data to a distance from its centroid space
- predict(X) : Predicts to which cluster X will belong to

In [ ]:
kmeans = KMeans(n_clusters=n_clusters,n_jobs=-1).fit(x)
labels = kmeans.labels_
silhouette = metrics.silhouette_score(x, labels)

### [Mean Shift](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.MeanShift.html)

![Mean shift search](https://cdn-images-1.medium.com/max/800/1*bkFlVrrm4HACGfUzeBnErw.gif)
![Mean shift algorithm](https://cdn-images-1.medium.com/max/800/1*vyz94J_76dsVToaa4VG1Zg.gif)

Mean shift algorithm :
   1. Arbitrarly defining n centroids
   2. Find all points lying in the neighbourhood of each centroids defined by the bandwith
   3. Putting the new centroids at the mean of all the points of a neighbourhood
   4. Repeating 2. and 3. until centroids converge<br>
Complexity = $O(n log(n))$

Parameters :
- bandwidth (float) : bandwidth that defines the neighbourhood, if not given estimated using sklearn.cluster.estimate_bandwidth (does not scale well)
- seeds (array) : seeds to use for initial centroids, if not given clustering.get_bin_seeds makes a seed grid using bandwidth as size.
- bin_seeding (boolean) : False (default) means all the seeds points are used, silenced if seeds are not given
- min_bin_freq (int) : Minimal number of points in the bin to induce a seed, default is 1
- cluster_all (boolean) : True (default) forces the clusterization of all points
- n_jobs (int) : lets you define how many CPU you want to use for computation, -1 means all, None (default) is equal to 1 by default

Attributes :
- cluster_centers_ (array) : gives the position of all your clusters, shape : [$n_{clusters}$ \, $n_{features}$]
- labels_ (list) : Labels of each point given to fit()

Methods :
- fit(X) : Runs the algorithm to find the clusters
- predict(X) : Predicts to which cluster X will belong to

### [Piping K-Means and Mean Shift](http://jamesxli.blogspot.com/2012/03/on-mean-shift-and-k-means-clustering.html)

By running a K-Means with a K bigger than the real number of clusters. You can produce the seeds for a Mean Shift.

![Piping K-means and Mean Shift](http://1.bp.blogspot.com/-4-yQSzGmCC8/T1aNsa6nJJI/AAAAAAAAH00/SWyAy7MC69M/s640/MapClustering3.png)

### Density Based Spatial Clustering of Applications with Noise ( [DBSCAN](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html) ) 

![DBSCAN illustration](https://cdn-images-1.medium.com/max/800/1*tc8UF-h0nQqUfLC8-0uInQ.gif)

DBSCAN algorithm : Here is [a nice visualization](https://www.naftaliharris.com/blog/visualizing-dbscan-clustering/)
   1. for a data point, get all the points that are within $ \epsilon $ reach
   2. if there are more then min_samples :
       - a cluster with all the points is formed and can be broaden with the other points
       - otherwise the point is tagged as noise
   3. repeat 1. and 2. for all the points<br>
Complexity = $O(n)$
   
Useful parameters :
- eps (float) : reach $\epsilon$ of the neighbourhood of a point
- min_samples (int) : Minimum number of points to declare a cluster
- metric (string) : option from [sklearn.metrics.pairwise_distances](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise_distances.html), [‘cityblock’, ‘cosine’, ‘euclidean’, ‘l1’, ‘l2’, ‘manhattan’]
- n_jobs (int) : lets you define how many CPU you want to use for computation, -1 means all, None (default) is equal to 1 by default

Attributes :
- core_sample_indices_ (array) : Indices of core samples.
- components_ (array) : Coordinates of each core sample found by training, shape = $[n_{core samples}\ , n_{features}]$
- labels_ (array) : Cluster labels for each point in the dataset given to fit(). Noisy samples are given the label -1.

Methods :
- fit(X) : Runs the algorithm to find the clusters
- fit_predict(X) : Runs the algorithm to find the clusters and returns the labels

In [ ]:
db = DBSCAN(eps=eps, min_samples=min_s).fit(X)
labels=db.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

### Ordering Points To Identify the Clustering Structure ( [OPTICS](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.OPTICS.html) ) 

OPTICS algorithm : [Here is a nice illustration](https://www.youtube.com/watch?v=8kJjgowewOs)
   1. Computing _Core Distances_ for each point of the dataset ($\epsilon$ of the neighbourhood with min_samples)
   2. Choosing a point, set i to 0
   3. Setting its $order=i$
   4. Setting the $reachability = max(Core \ Distance \ , \   distance_{closest \ neighbour})$
   5. Incrementing $i=i+1$ 
   6. Repeating 3. 4. and 5. using the closest neighbour<br>
Complexity = $O(n^2)$

You then get a reachability plot :
![Reachability plot](https://scikit-learn.org/stable/_images/sphx_glr_plot_optics_001.png)


A valley represents a cluster. A valley into another valley represents a nested cluster. <br>
You can represent the behaviour of DBSCAN by putting a threshold at a certain level and seperating the cluster everytime the reachability of the next point (which is the closest remember) is exceeding the threshold

Useful parameters :
- max_eps (float) : maximum reachability to look for
- min_samples (int) : Minimum number of points to declare a cluster
- metric (string) : option from [sklearn.metrics.pairwise_distances](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise_distances.html), [‘cityblock’, ‘cosine’, ‘euclidean’, ‘l1’, ‘l2’, ‘manhattan’]
- cluster_method (string) : ['xi', 'dbscan'] method to create the cluster from the reachability plot
- eps (float) : if cluster_method='dbscan', threshold to declare a point as noise
- xi (float) : if cluster_method='xi', determines steepness to declare further points as noise $\frac{x_i}{x_{i+1}}<1-xi$
- n_jobs (int) : lets you define how many CPU you want to use for computation, -1 means all, None (default) is equal to 1 by default

USeful Attributes :
- labels_ (array) : Cluster labels for each point in the dataset given to fit(). Noisy samples are given the label -1.
- reachability_ (array) : Reachability distances per sample, indexed by object order. 
- ordering_ (array) : The cluster ordered list of sample indices. Use clust.reachability_[clust.ordering_] to access in cluster order.

Methods :
- fit(X) : Runs the algorithm to find the clusters
- fit_predict(X) : Runs the algorithm to find the clusters and returns the labels

In [ ]:
min_samples=100
title="reachability plot given a minimum samples of "+ str(min_samples)

clust = OPTICS(min_samples=min_samples)
clust.fit(x)

reachability = clust.reachability_[clust.ordering_]

plt.plot(reachability)
plt.title(title)
plt.xlabel("order")
plt.ylabel("reachability")
plt.ylim(top=reachability[1:].mean()+3*reachability[1:].std())
plt.show()

### [Expectation Maximization ( EM ) using Gaussian Mixture](https://scikit-learn.org/stable/modules/generated/sklearn.mixture.GaussianMixture.html)

![EM Clustering viz](https://cdn-images-1.medium.com/max/800/1*OyXgise21a23D5JCss8Tlg.gif)

EM algorithm :
   1. Set K Gaussian with random means and covariance matrices
   2. Compute for every point the probability that it belongs to the gaussian
   3. Put the mean to the weighted (by the probability of each points) mean of all points and adapt the covariance to maximize the likelihood of the Gaussian given the data
   4. Repeat 2. and 3. until convergence<br>
   Complexity = $O(n)$

### Hierarchical Agglomerative Clustering ( [HAC](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html) )

This clustering is a bottom up agglomerative algorithm (starts from a unique point and ends on a big cluster containing every points)

![Hierarchical Clustering viz](https://cdn-images-1.medium.com/max/800/1*ET8kCcPpr893vNZFs8j4xg.gif)

HAC algorithm :
   1. Each point is treated as a cluster and every distance to each cluster is computed
   2. The two clusters with the smaller distance are then merged into a unique cluster
   3. Repeat until there is only one cluster left<br>
   Complexity = $O(n^3)$



   


# [Model Selection](https://scikit-learn.org/stable/tutorial/statistical_inference/model_selection.html)

## [Grid Search](https://scikit-learn.org/stable/modules/grid_search.html)

A Grid Search is used to fine the best hyperparameter for your model.

* a parameter space (which parameters of your are you gonna tune)
* a method for searching and sampling candidates (which values are the parameters gonna take)
* an estimator (what regressor or classifier will make the predictions)
* a score function (how are you gonna measure which model is better)
* a cross validation scheme (for unbiased estimator you have to cross validate)

### Defining a grid of parameters

Here is a standard parameter grid for a kernel PCA decomposition problem

In [ ]:
param_grid = [
    {'pca__kernel': ['linear','cosine'],
     'pca__n_components':[1,2,3,4,5,6]},
    {'pca__kernel': ['rbf'], 
     'pca__gamma':[10**-6, 10**-5, 10**-4, 0.001, 0.01, 0.1, 1, 10],
     'pca__n_components':[1,2,3,4,5,6]},
    {'pca__kernel': ['sigmoid'],
     'pca__gamma':[-10**-6,-10**-5,-10**-4,-0.001,-0.01,-0.1,-1,-10,
                    10**-6, 10**-5, 10**-4, 0.001, 0.01, 0.1, 1, 10],
     'pca__coef0':[-100,-10,-5,-1,-0.1,0,
                    100, 10, 5, 1, 0.1],
     'pca__n_components':[1,2,3,4,5,6]},
    {'pca__kernel': ['poly'],
     'pca__gamma':[10**-6, 10**-5, 10**-4, 0.001, 0.01, 0.1, 1, 10],
     'pca__coef0':[-100,-10,-5,-1,-0.1,0,
                    100, 10, 5, 1, 0.1],
     'pca__degree':[-5,-4,-3,-2,-1,-0.5,0.5,2,3,4,5],
     'pca__n_components':[1,2,3,4,5,6]},
 ]

### Applying the Grid Search

[GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)
lets you choose several of the options you want for your Grid Search

* estimator (object with a score function)
* param_grid (dict)
* n_jobs (int) : number of jobs to run in parallel : -1 sets maximum
* cv (int) : number of fold for the Kfold or Stratified Kfold (default) or cv method
* verbose (int) : 0 (no output) 1(some outout) 2(every CV time output) 3(CV time + score output)

Useful attribute :
* cv_results : Dict with results
* best_estimator_ : object estimator with the parameters that yielded the best score

Methods :
* fit(X,y) : Runs fits for all the parameters
* transform(X) : Runs transform of X for the best estimator
* predict(X) : Runs predict of X using the best estimator

## [Cross validation](https://scikit-learn.org/stable/modules/cross_validation.html)

In [ ]:
import numpy as np
from sklearn.model_selection import KFold

X = np.array([[1, 2], [3, 4], [5, 6], [7, 8], [9,10], [11,12]])  #your EARLY dataset
y = np.array([0, 1, 2, 3, 4, 5])                              #your PREDICTED dataset
kf = KFold(n_splits=3)   #do a 3 fold
print(X.shape, y.shape)
scores=list()

for train_index, test_index in kf.split(X,y):
    print("TRAINindex:", train_index, "TESTindex:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    print("TrainSet: \n", X_train, "\n", y_train,"\n TestSet: \n",X_test, "\n",y_test)
    
    # DEFINE A MODEL HERE
    
    # FIT A MODEL HERE ON X_TRAIN + y_train
    
    # EVALUATE MODEL HERE X_TEST + y_test
    
    # STORE THE RESULTS in a list scores=list() scores.append(accuracy,loss)
    
print('Estimated Accuracy %.3f (%.3f)' % (np.mean(scores), np.std(scores)))
